In [34]:
import tensorflow as tf
import numpy as np


# to reshape (5,1) back to (5,) you can use ...
#shape tests
#
X_XOR=np.array([[0,0],[0,1],[1,0],[1,1]])
Y_XOR = np.array([0,1,1,0])

tf.reset_default_graph()
#why do the above? get memory leak, keeps on adding variables and allocating space. 
#unstack:1 will get incremented to unstack:2, etc...everytime you run this you get new allocations
#user program can cause OOM. 

print("np shape:",X_XOR.shape)

X1a = tf.placeholder(tf.float32, shape=([None,len(X_XOR)]), name="X1a")
X2a = tf.constant(X_XOR,name="X2a")
normal_dist = tf.truncated_normal(shape=[10,1],mean=0.0, stddev=.01,dtype=tf.float32, name='normaldist')
weight = tf.Variable(normal_dist, name='weight')
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(X1a)
    print(X2a)

X1a_static_shape = X1a.shape.as_list()
X1a_dynamic_shape = tf.unstack(tf.shape(X1a))

X2a_static_shape = X1a.shape.as_list()
X2a_dynamic_shape = tf.unstack(tf.shape(X1a))

weight_static_shape = weight.shape.as_list()
weight_dynamic_shape = tf.unstack(tf.shape(weight))

print("X1a_static_shape:",X1a_static_shape, "X1a_dynamic_shape:",X1a_dynamic_shape)
print("X2a_static_shape:",X2a_static_shape, "X2a_dynamic_shape:",X2a_dynamic_shape)

print("weight:",weight_static_shape, "weight_dynamic_shape:",weight_dynamic_shape)


np shape: (4, 2)
Tensor("X1a:0", shape=(?, 4), dtype=float32)
Tensor("X2a:0", shape=(4, 2), dtype=int64)
X1a_static_shape: [None, 4] X1a_dynamic_shape: [<tf.Tensor 'unstack:0' shape=() dtype=int32>, <tf.Tensor 'unstack:1' shape=() dtype=int32>]
X2a_static_shape: [None, 4] X2a_dynamic_shape: [<tf.Tensor 'unstack_1:0' shape=() dtype=int32>, <tf.Tensor 'unstack_1:1' shape=() dtype=int32>]
weight: [10, 1] weight_dynamic_shape: [<tf.Tensor 'unstack_2:0' shape=() dtype=int32>, <tf.Tensor 'unstack_2:1' shape=() dtype=int32>]


In [18]:
import tensorflow as tf


var0 = tf.Variable([1.0, 2.0], dtype=tf.float32)
var1 = tf.Variable([3.0, 4.0], dtype=tf.float32)
grads0 = tf.constant([0.1, 0.1], dtype=tf.float32)
grads1 = tf.constant([0.01, 0.01], dtype=tf.float32)
with tf.Session() as sess:
    sgd_op = tf.train.GradientDescentOptimizer(3.0).apply_gradients(
        zip([grads0, grads1], [var0, var1]))
    tf.global_variables_initializer().run()
    # Fetch params to validate initial values
    #self.assertAllCloseAccordingToType([3.0, 4.0], var1.eval())
    # Run 1 step of sgd
    sgd_op.run()
    print("var0:",sess.run(var0))
    print("var1:",sess.run(var1))
    print("grads0:",sess.run(grads0))
    print("grads1:",sess.run(grads1))
    
#this is funny. The learning rate is 3.0 so 3.0*0.1 is .3. But when you do the multiply
#you dont get .7. Is a little off bc rounding error. 
#


var0: [ 0.69999999  1.70000005]
var1: [ 2.97000003  3.97000003]
grads0: [ 0.1  0.1]
grads1: [ 0.01  0.01]


In [ ]:
# updating a variable; can't update a tensor b/c it is immutable. 
#a scatter updates only the first dimension in an array. 

# this a better style, define the graph first
#
g = tf.Graph()
with g.as_default():
    a = tf.Variable(initial_value=[[0, 0, 0, 0],[0, 0, 0, 0]])
    b = tf.scatter_update(a, [0, 1], [[1, 0, 0, 0], [1, 0, 0, 0]])

with tf.Session(graph=g) as sess:
    sess.run(tf.initialize_all_variables())
    print sess.run(a)
    print sess.run(b)

#scatter used to distribute cells to larger/smaller matrices
#this for distributed copy


In [ ]:
#tf practice; from the Udacity website
# do not do this
#learning rate

import tensorflow as tf

learning_rate = .001
training_epocs = 20
batch_size = 20
display_step = 1
n_input = 784
n_classes = 10
n_hidden_layer = 256

#save_file = './model.ckpt'

#Weights and biases
weights = {
    'hidden_layer' : tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal[n_classes])    
}

biases = {
    'hidden_layer' : tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal[n_classes])
}


#Input
x = tf.placeholder("float", [None,28,28,1])
y = tf.placeholder("float", [None,n_classes])

x_flat = tf.reshape(x,[-1,n_input]) #this is different from numpy reshape, -1 vs. none reshape

layer_1 = tf.add(tf.matmul(x_flat,weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu('layer_1')
logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

#optimizer

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

#session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer,feed_dict={x:batch_x, y:batch_y})
            

